[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://github.com/saeid-uot/3253-Machine-Learning/blob/main/GitHubCodeRepo/Week_1/People_SatisfactionGDP.ipynb)


<a href="https://colab.research.google.com/github/googlecolab/colabtools/blob/master/notebooks/colab-github-demo.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Predict People Satisfaction Across the Globe

**Objective:**
We would like to build a model that predicts satisfaction score for people of different countries given their country GDP.

#Note 1: How to enable code completion:

Tools menu ==> click on settings ==> Editor ==> Enable "Automatically trigger code completions"




#Note 2: Instructions to create a copy of this notebook for youtself

You do not have write access to this notebook.

* From Menu bar, Go to File, 
* Select "Save a copy in my Drive"
* Navigate to Google Drive
* Find a folder named "Collab Notebook" and open it to find your notebook. 
* Rename it and start making changes.

**Note:** If there is any file you should read in your code, make sure you copy the file from instructor folder to your own Gdrive by following below steps:

* Right clicking on the file name 
* Select "Make a copy"
* Click on the new file
* Move it to desired folder, preferrably where you have your notebook

# Download Dataset

Download the Better Life Index data (latest edition, currently it is 2017) from the [OECD’s website](http://homl.info/4) as well as stats about GDP per capita from the [IMF’s website](http://homl.info/5). Then you join the tables and sort by GDP per capita. 

# Import Dataset to Google Colab

1. Download CSV and XLS files to your computer
2. Upload them to your Google Drive
3. Open the CSV files using Google Sheets so Google will create the dataset in format of Google Sheets
4. You can remove CSV and XLS files from your drive
5. Use the step by step guide from [here](https://colab.research.google.com/notebooks/io.ipynb#scrollTo=vz-jH8T_Uk2c) and scroll down to **" Google Sheets" ** cell to import data into dataframe

NOTE: After creating Google Sheet into your Drive, make sure you are converting Column 2015 to 0.00 format before importing it into Colab  otherwise Google will import it as a string and you will have hard time to clean the data




In [ ]:
#get file from google drive
%%time
!pip install googledrivedownloader #black magic
from google_drive_downloader import GoogleDriveDownloader as gdd
                                            
# gdd.download_file_from_google_drive(file_id="1AavEtw38uMBhcDLmejYU_L8LVbbrWtlh",
#                                     dest_path="./BLI_30012019054825599.xlsx",
#                                     unzip=False)

gdd.download_file_from_google_drive(file_id="1N8nT2DvzCV8e-5T01u5gAtWVf2ghl_mc",
                                    dest_path="./BLI.xlsx",
                                    unzip=False)

gdd.download_file_from_google_drive(file_id="1IaJWQNnk6QdP7VjiovI3v4WobOnxnQdE",
                                    dest_path="./WEO_Data.xlsx",
                                    unzip=False)

CPU times: user 42.5 ms, sys: 9.3 ms, total: 51.8 ms
Wall time: 3.75 s


In [ ]:
import pandas as pd
from IPython.display import display

#load excel as dataframe
bli = pd.read_excel("BLI.xlsx", sheet_name=0)
display(bli) #inspect if the dataframe is loaded correctly

# Remove rows where inequality has values other than TOT
bli = bli[bli["INEQUALITY"]=="TOT"]

# Reformat data based on "indicator column"
bli = bli.pivot(index="Country", columns="Indicator", values="Value")

bli["Life satisfaction"].head()

,LOCATION,Country,INDICATOR,Indicator,MEASURE,Measure,INEQUALITY,Inequality,Unit Code,Unit,PowerCode Code,PowerCode,Reference Period Code,Reference Period,Value,Flag Codes,Flags
0,AUS,Australia,JE_LMIS,Labour market insecurity,L,Value,TOT,Total,PC,Percentage,0.0,Units,NaN,NaN,4.300000,NaN,NaN
1,AUS,Australia,CG_SENG,Stakeholder engagement for developing regulations,L,Value,TOT,Total,AVSCORE,Average score,0.0,Units,NaN,NaN,2.700000,NaN,NaN
2,AUS,Australia,CG_SENG,Stakeholder engagement for developing regulations,L,Value,MN,Men,AVSCORE,Average score,0.0,Units,NaN,NaN,2.700000,E,Estimated value
3,AUS,Australia,CG_SENG,Stakeholder engagement for developing regulations,L,Value,WMN,Women,AVSCORE,Average score,0.0,Units,NaN,NaN,2.700000,E,Estimated value
4,AUS,Australia,PS_FSAFEN,Feeling safe walking alone at night,L,Value,TOT,Total,PC,Percentage,0.0,Units,NaN,NaN,63.600000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3394,USA,United States,WL_EWLH,Employees working very long hours,L,Value,WMN,Women,PC,Percentage,0.0,Units,NaN,NaN,6.860000,NaN,NaN
3395,USA,United States,WL_TNOW,Time devoted to leisure and personal care,L,Value,TOT,Total,HOUR,Hours,0.0,Units,NaN,NaN,14.440000,NaN,NaN
3396,USA,United States,WL_TNOW,Time devoted to leisure and personal care,L,Value,MN,Men,HOUR,Hours,0.0,Units,NaN,NaN,14.480000,NaN,NaN
3397,USA,United States,WL_TNOW,Time devoted to leisure and personal care,L,Value,WMN,Women,HOUR,Hours,0.0,Units,NaN,NaN,14.390000,NaN,NaN


Country
Australia    7.3
Austria      7.0
Belgium      6.9
Brazil       6.6
Canada       7.3
Name: Life satisfaction, dtype: float64

In [ ]:
bli[0:5]

Indicator,Air pollution,Dwellings without basic facilities,Educational attainment,Employees working very long hours,Employment rate,Feeling safe walking alone at night,Homicide rate,Household net adjusted disposable income,Household net financial wealth,Housing expenditure,Labour market insecurity,Life expectancy,Life satisfaction,Long-term unemployment rate,Personal earnings,Quality of support network,Rooms per person,Self-reported health,Stakeholder engagement for developing regulations,Student skills,Time devoted to leisure and personal care,Voter turnout,Water quality,Years in education
Country,,,,,,,,,,,,,,,,,,,,,,,,
Australia,5.0,1.1,80.0,13.20,72.0,63.6,1.0,33417.0,57462.0,20.0,4.3,82.5,7.3,1.36,52063.0,94.0,2.3,85.0,2.7,502.0,14.35,91.0,92.0,21.2
Austria,16.0,1.0,85.0,6.78,72.0,80.7,0.4,32544.0,59574.0,21.0,2.7,81.3,7.0,1.94,48295.0,92.0,1.6,70.0,1.3,492.0,14.55,75.0,93.0,17.1
Belgium,15.0,2.3,75.0,4.31,62.0,70.7,1.0,29968.0,104084.0,21.0,4.8,81.1,6.9,3.98,49587.0,92.0,2.2,75.0,2.2,503.0,15.77,89.0,84.0,18.2
Brazil,10.0,6.7,49.0,7.15,64.0,37.3,27.6,12227.0,7102.0,20.0,4.9,74.7,6.6,3.37,14024.0,90.0,0.8,70.0,2.2,395.0,14.45,79.0,72.0,15.9
Canada,7.0,0.2,91.0,3.73,73.0,80.9,1.4,29850.0,85758.0,22.0,3.9,81.5,7.3,0.81,48403.0,93.0,2.5,88.0,3.0,523.0,14.41,68.0,91.0,16.7


In [ ]:
#load excel as dataframe
weo = pd.read_excel("WEO_Data.xlsx", sheet_name=0)
display(weo) #inspect if the dataframe is loaded correctly

# Drop the header row from data
weo = weo.reindex(weo.index.drop(0))

# 1- Select only Country name and 2015 
# 2- then rename it to GDP Per capita
weo = weo[['Country','2015']].rename(columns={'2015':'GDP per capita'})

# Set Country as index column
# Inplace command, will replace the results of command into the same DF
weo.set_index("Country", inplace=True)

#weo.drop_duplicates(inplace=True)
#Print top 5 rows
weo.head()


,Country,Subject Descriptor,Units,Scale,Country/Series-specific Notes,2015,Estimates Start After
0,Afghanistan,"Gross domestic product per capita, current prices",U.S. dollars,Units,"See notes for: Gross domestic product, curren...",599.994,2013.0
1,Albania,"Gross domestic product per capita, current prices",U.S. dollars,Units,"See notes for: Gross domestic product, curren...",3995.380,2010.0
2,Algeria,"Gross domestic product per capita, current prices",U.S. dollars,Units,"See notes for: Gross domestic product, curren...",4318.140,2014.0
3,Angola,"Gross domestic product per capita, current prices",U.S. dollars,Units,"See notes for: Gross domestic product, curren...",4100.320,2014.0
4,Antigua and Barbuda,"Gross domestic product per capita, current prices",U.S. dollars,Units,"See notes for: Gross domestic product, curren...",14414.300,2011.0
...,...,...,...,...,...,...,...
186,Yemen,"Gross domestic product per capita, current prices",U.S. dollars,Units,"See notes for: Gross domestic product, curren...",1302.940,2008.0
187,Zambia,"Gross domestic product per capita, current prices",U.S. dollars,Units,"See notes for: Gross domestic product, curren...",1350.150,2010.0
188,Zimbabwe,"Gross domestic product per capita, current prices",U.S. dollars,Units,"See notes for: Gross domestic product, curren...",1064.350,2012.0
189,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,GDP per capita
Country,
Albania,3995.38
Algeria,4318.14
Angola,4100.32
Antigua and Barbuda,14414.30
Argentina,13588.85


# Merge/Join dataset

In [ ]:
# Now merge BLI and WEO datasets
df = pd.merge(left = weo, right = bli, left_index=True, right_index=True)

#sort the dataframe by GPD per capita
df.sort_values(by="GDP per capita", inplace=True)
df.head()


,GDP per capita,Air pollution,Dwellings without basic facilities,Educational attainment,Employees working very long hours,Employment rate,Feeling safe walking alone at night,Homicide rate,Household net adjusted disposable income,Household net financial wealth,Housing expenditure,Labour market insecurity,Life expectancy,Life satisfaction,Long-term unemployment rate,Personal earnings,Quality of support network,Rooms per person,Self-reported health,Stakeholder engagement for developing regulations,Student skills,Time devoted to leisure and personal care,Voter turnout,Water quality,Years in education
Country,,,,,,,,,,,,,,,,,,,,,,,,,
South Africa,5694.57,22.0,37.0,43.0,18.68,43.0,36.1,10.0,10872.0,17042.0,18.0,26.5,57.4,4.8,15.57,11554.0,88.0,0.7,67.0,1.6,391.0,14.73,73.0,69.0,15.3
Brazil,8670.00,10.0,6.7,49.0,7.15,64.0,37.3,27.6,12227.0,7102.0,20.0,4.9,74.7,6.6,3.37,14024.0,90.0,0.8,70.0,2.2,395.0,14.45,79.0,72.0,15.9
Mexico,9009.28,16.0,4.2,37.0,29.48,61.0,45.9,17.9,13891.0,4750.0,21.0,4.6,75.0,6.6,0.08,15311.0,80.0,1.0,66.0,3.5,416.0,12.74,63.0,67.0,14.8
Russia,9054.91,15.0,13.8,95.0,0.16,70.0,52.2,11.3,16657.0,2260.0,19.0,3.6,71.3,6.0,1.64,22101.0,90.0,1.0,43.0,0.8,492.0,14.90,65.0,54.0,16.1
Turkey,9437.37,20.0,6.5,39.0,33.77,51.0,60.6,1.7,17067.0,4429.0,20.0,13.0,78.0,5.5,2.24,22848.0,86.0,1.0,66.0,2.1,425.0,12.59,85.0,63.0,17.9


In [ ]:
df.iloc[1]

GDP per capita                                        8670.00
Air pollution                                           10.00
Dwellings without basic facilities                       6.70
Educational attainment                                  49.00
Employees working very long hours                        7.15
Employment rate                                         64.00
Feeling safe walking alone at night                     37.30
Homicide rate                                           27.60
Household net adjusted disposable income             12227.00
Household net financial wealth                        7102.00
Housing expenditure                                     20.00
Labour market insecurity                                 4.90
Life expectancy                                         74.70
Life satisfaction                                        6.60
Long-term unemployment rate                              3.37
Personal earnings                                    14024.00
Quality 

## Split dataset into Train & Test

In [ ]:
# Below snippet is the most basic way of splitting data in Python; It is for illustration only and in 
# later sections we will use proper library from Sklearn to split the data
test_indices = [0, 1, 6, 8, 33, 34, 35]

#define the indices of the training set by substracting the test indices from all indices
train_indices = list(set(range(36)) - set(test_indices))

#use the above indices to select parts of the dataframe as the training set and the test set
train = df[["GDP per capita", 'Life satisfaction']].iloc[train_indices]
test = df[["GDP per capita", 'Life satisfaction']].iloc[test_indices]

In [ ]:
train.head()

,GDP per capita,Life satisfaction
Country,,
Mexico,9009.28,6.6
Russia,9054.91,6.0
Turkey,9437.37,5.5
Hungary,12239.89,5.3
Chile,13340.91,6.7


In [ ]:
# Code example
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn.linear_model

# Prepare the data
X = np.c_[train["GDP per capita"]]
y = np.c_[train["Life satisfaction"]]

In [ ]:
type(y)

numpy.ndarray

In [ ]:
#Lets look at what is inside X and y. Print first 5 records
X[:5], y[:5]

(array([[ 9009.28],
        [ 9054.91],
        [ 9437.37],
        [12239.89],
        [13340.91]]), array([[6.6],
        [6. ],
        [5.5],
        [5.3],
        [6.7]]))

# ML Starts Now
## Define a model with default values

In [ ]:
# Select a basic linear model without setting any parameter (nothing inside paranthesis below)
model = sklearn.linear_model.LinearRegression()

# See the model for yourself
model

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

## Start training the model using X and y

In [ ]:
# Train the model
model.fit(X, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [ ]:
model.coef_

array([[3.93552544e-05]])

In [ ]:
model.intercept_

array([5.29552139])

## Do prediction on test data

In [ ]:
# Make a prediction for Cyprus
X_new = [[17770]]  # Cyprus' GDP per capita
print(model.predict(X_new)) # outputs [[5.95199478]]

[[5.99486426]]


In [ ]:
# Make a prediction for our test data
pred = model.predict(test['GDP per capita'].values.reshape(-1,1))

In [ ]:
pred

array([[5.51963264],
       [5.63673145],
       [5.78727828],
       [5.83148368],
       [7.34648781],
       [7.49174923],
       [8.24016457]])